In [1]:
import subprocess
import sys

# List of required packages
required_packages = [
    "pickle", "numpy", "keybert", "tqdm", "pandas", "torch", "sentence_transformers"
]

def install_package(package):
    """Installs a package using pip if it's not already installed."""
    try:
        __import__(package)
        print(f"{package} is already installed.")
    except ImportError:
        print(f"Installing {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# Check and install missing packages
for package in required_packages:
    install_package(package)

pickle is already installed.
numpy is already installed.


C:\Users\aless\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\aless\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\utils\_param_validation.py:11: UserWarning: A NumPy version >=1.22.4 and <2.3.0 is required for this version of SciPy (detected version 2.3.1)
  from scipy.sparse import csr_matrix, issparse


keybert is already installed.
tqdm is already installed.
pandas is already installed.
torch is already installed.
sentence_transformers is already installed.


In [2]:
import pickle
import pandas as pd
import numpy as np

from sentence_transformers import SentenceTransformer
from KeyBertMetadata import KeyBERTMetadata

import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

import warnings
warnings.filterwarnings('ignore')

In [3]:
movie_titles = ['../Dataset/Reviews_By_Movie/GoodBadUgly.pkl',
                '../Dataset/Reviews_By_Movie/HarryPotter.pkl',
                '../Dataset/Reviews_By_Movie/IndianaJones.pkl',
                '../Dataset/Reviews_By_Movie/LaLaLand.pkl',
                '../Dataset\Reviews_By_Movie\Oppenheimer.pkl',
                '../Dataset/Reviews_By_Movie/Parasite.pkl',
                '../Dataset/Reviews_By_Movie/SW_Episode1.pkl',
                '../Dataset/Reviews_By_Movie/SW_Episode2.pkl',
                '../Dataset/Reviews_By_Movie/SW_Episode3.pkl',
                '../Dataset/Reviews_By_Movie/SW_Episode4.pkl',
                '../Dataset/Reviews_By_Movie/SW_Episode5.pkl',
                '../Dataset/Reviews_By_Movie/SW_Episode6.pkl',
                '../Dataset/Reviews_By_Movie/SW_Episode7.pkl',
                '../Dataset/Reviews_By_Movie/SW_Episode8.pkl',
                '../Dataset/Reviews_By_Movie/SW_Episode9.pkl']

model = KeyBERTMetadata(model=SentenceTransformer("all-MiniLM-L6-v2"))
df_subset_length = 2000

for file_path in movie_titles:
    with open(file_path, 'rb') as file:
        reviews_df = pickle.load(file)

    # Assuming the reviews_df has a 'Movie_Title' column
    movie_title = reviews_df['Movie_Title'].iloc[0]
    print("Loaded dataset: " + movie_title)

    df_subset = reviews_df.head(df_subset_length)

    metadata = model.extract_metadata(df_subset, alpha=0.3)
    docs = df_subset["Preprocessed_Review"].astype(str).tolist()


    # Estrai document embeddings con metadati
    doc_embeddings, _ = model.extract_embeddings_mean(
        docs,
        metadata=metadata,
        optional_pruning=False,
        keyphrase_ngram_range=(1, 2),
        stop_words='english'
    )

    # PCA per analisi dimensionale
    pca = PCA(n_components=390)
    pca_result = pca.fit_transform(doc_embeddings)


    # Ogni riga di pca.components_ è una componente principale
    # Ogni colonna corrisponde a una dimensione dell'embedding originale
    loadings = pca.components_

    # Dimensione totale dell'embedding (inclusi i metadati)
    n_features = doc_embeddings.shape[1]

    # Calcoliamo la "forza media" per ciascuna dimensione
    avg_contribution = np.mean(np.abs(loadings), axis=0)

    # Visualizziamo le dimensioni più influenti
    top_indices = np.argsort(avg_contribution)[::-1][:]

    top_indices_df = pd.DataFrame({
    'Dimension': top_indices,
    'Average Contribution': avg_contribution[top_indices]
    })

    display(top_indices_df)

    # Indici delle dimensioni dei metadati
    #meta_indices = [381, 382, 383, 384, 385, 386] #da usare con pruning attivo
    meta_indices = [384, 385, 386, 387, 388, 389] #da usare con pruning NON attivo

    # nella variabile title inserisci il titolo del fil trattato in parasite reviews df['movie title']
    title = reviews_df['Movie_Title'].iloc[0]

    print(f"{title}  {len(df_subset)} reviews")

    for idx in meta_indices:
        # Stampa anche la posizione in top indices dei metaindices
        position = np.where(top_indices == idx)[0][0]
        print(f"Dimensione {idx} (metadato): contributo medio {avg_contribution[idx]:.6f}, posizione in top indices: {position+1}/{len(top_indices)}")
        
        #print(f"Dimensione {idx} (metadato): contributo medio {avg_contribution[idx]:.6f}")
    
    print("\n")


    


Loaded dataset: The Good, the Bad and the Ugly


,Dimension,Average Contribution
0,270,0.041913
1,74,0.041889
2,327,0.041649
3,307,0.041560
4,362,0.041495
...,...,...
385,384,0.015129
386,388,0.011677
387,127,0.002570
388,223,0.002570


The Good, the Bad and the Ugly  1429 reviews
Dimensione 384 (metadato): contributo medio 0.015129, posizione in top indices: 386/390
Dimensione 385 (metadato): contributo medio 0.026082, posizione in top indices: 383/390
Dimensione 386 (metadato): contributo medio 0.020187, posizione in top indices: 384/390
Dimensione 387 (metadato): contributo medio 0.029277, posizione in top indices: 381/390
Dimensione 388 (metadato): contributo medio 0.011677, posizione in top indices: 387/390
Dimensione 389 (metadato): contributo medio 0.017915, posizione in top indices: 385/390


Loaded dataset: Harry Potter and the Sorcerer's Stone


,Dimension,Average Contribution
0,360,0.041424
1,293,0.041406
2,19,0.041390
3,366,0.041323
4,185,0.041304
...,...,...
385,388,0.016798
386,389,0.013874
387,127,0.002568
388,223,0.002568


Harry Potter and the Sorcerer's Stone  2000 reviews
Dimensione 384 (metadato): contributo medio 0.017813, posizione in top indices: 385/390
Dimensione 385 (metadato): contributo medio 0.026957, posizione in top indices: 383/390
Dimensione 386 (metadato): contributo medio 0.025239, posizione in top indices: 384/390
Dimensione 387 (metadato): contributo medio 0.028832, posizione in top indices: 381/390
Dimensione 388 (metadato): contributo medio 0.016798, posizione in top indices: 386/390
Dimensione 389 (metadato): contributo medio 0.013874, posizione in top indices: 387/390


Loaded dataset: Raiders of the Lost Ark


...

Star Wars: Episode VII - The Force Awakens  2000 reviews
Dimensione 384 (metadato): contributo medio 0.025585, posizione in top indices: 382/390
Dimensione 385 (metadato): contributo medio 0.022536, posizione in top indices: 384/390
Dimensione 386 (metadato): contributo medio 0.025366, posizione in top indices: 383/390
Dimensione 387 (metadato): contributo medio 0.018786, posizione in top indices: 385/390
Dimensione 388 (metadato): contributo medio 0.011500, posizione in top indices: 386/390
Dimensione 389 (metadato): contributo medio 0.008072, posizione in top indices: 387/390


Loaded dataset: Star Wars: Episode VIII - The Last Jedi


,Dimension,Average Contribution
0,50,0.041508
1,11,0.041459
2,171,0.041409
3,267,0.041393
4,377,0.041357
...,...,...
385,389,0.013015
386,388,0.012915
387,127,0.002820
388,223,0.002820


Star Wars: Episode VIII - The Last Jedi  2000 reviews
Dimensione 384 (metadato): contributo medio 0.019556, posizione in top indices: 385/390
Dimensione 385 (metadato): contributo medio 0.022094, posizione in top indices: 384/390
Dimensione 386 (metadato): contributo medio 0.023111, posizione in top indices: 383/390
Dimensione 387 (metadato): contributo medio 0.024011, posizione in top indices: 382/390
Dimensione 388 (metadato): contributo medio 0.012915, posizione in top indices: 387/390
Dimensione 389 (metadato): contributo medio 0.013015, posizione in top indices: 386/390


Loaded dataset: Star Wars: Episode IX - The Rise of Skywalker


,Dimension,Average Contribution
0,271,0.042068
1,291,0.041695
2,181,0.041693
3,176,0.041632
4,144,0.041541
...,...,...
385,388,0.014714
386,389,0.010343
387,127,0.002590
388,223,0.002590


Star Wars: Episode IX - The Rise of Skywalker  2000 reviews
Dimensione 384 (metadato): contributo medio 0.024977, posizione in top indices: 384/390
Dimensione 385 (metadato): contributo medio 0.029286, posizione in top indices: 381/390
Dimensione 386 (metadato): contributo medio 0.026753, posizione in top indices: 383/390
Dimensione 387 (metadato): contributo medio 0.018034, posizione in top indices: 385/390
Dimensione 388 (metadato): contributo medio 0.014714, posizione in top indices: 386/390
Dimensione 389 (metadato): contributo medio 0.010343, posizione in top indices: 387/390


